In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import hyperspy.api as hs
import os

from tqdm.notebook import tqdm
from Gaen_packages import ImageTreatment_and_interfaces as ii

No scalebar avalable


# Navigation Algorithm

This algorithm determines the navigation path for detecting nanoparticles. The direction of movement is guided by the probability of encountering nanoparticles at the next step. This probability is provided by our model, which compares the likelihood of finding nanoparticles in different directions and selects the direction with the highest probability. Once the probability reaches a predefined threshold, the system will capture an image of the corresponding region.

In [ ]:
#settings
size_detection = 256
n = 32 #step movement
(i,j)= (1024/2 , 1024/2) #starting point for the detection algorith 
green = (0, 255, 0) 
red = (0, 0, 255) 
thickness = 2

In [ ]:
def find_np(p_np, fov):
"""
This algorithm navigates through the sample and detects and strores the regions where we encounter nanoparticles.
prov_mov = probability of detecting nanoparticles given by the model.
""" 
    #current
    scanning_m_current = M[i-fov: i, j:j+fov]
    prob_mov_current = prob_mov(scanning_m_current)
    
    #Direction determination: by calculating the highest probability where we can find nanoparticles, the algorithm will decide the best path.
    scanning_m_u = M[i-fov+n:i+n, j:j+fov]
    prob_mov_u = prob_mov(scanning_m_u)
    
    scanning_m_d = M[-i-fov-n:i-n, j:j+fov]
    prob_mov_d = prob_mov(scanning_m_d)
    
    scanning_m_r = M[i-fov:i, j+n:j+fov+n]
    prob_mov_r = prob_mov(scanning_m_r)
    
    scannig_m_l = M[i-fov:i, -j-fov-n:j-n]
    prob_mov_l = prob_mov(scannig_m_l)
    
    prob = [prob_mov_r, prob_mov_l, prob_mov_u, prob_mov_d]
    max_prob = np.max(prob)

    #Acquiring the image of the nanoparticle when the surroundings haven't higher porbability of finding a nanoparticle than the current posistion
    if max_prob <= prob_mov_current: 
        Append_coordinates (c_nanoparticle, i, j)
        nanoparticle_images.append(scanning_m_current)
    
    #Displacement
    if max_prob == prob_mov_r: #go to the right
        if Border_detection (i+fov+n, i-fov+n, j+fov, j-fov, dimension_right, dimension_up) == False:
            i,j = i+n, j
            Append_coordinates (c_displacement, i, j)
        else:

        
    elif max_prob == prob_mov_l: #go to the left
        if Border_detection (i+fov-n, -i-fov-n, j+fov, j-fov, dimension_right, dimension_up) == False:
            i,j = i-n, j
            Append_coordinates (c_displacement, i, j)
        else:

        
    elif max_prob == prob_mov_u: #go up
        if Border_detection (i+fov, i-fov, j+fov+n, -j-fov+n, dimension_right, dimension_up) == False:
            i,j = i, j+n
            Append_coordinates (c_displacement, i, j)
        else:

            
    elif max_prob == prob_mov_d: #go down
        if Border_detection (i+fov, i-fov, j+fov-n, -j-fov-n, dimension_right, dimension_up) == False:
            i,j = i, j-n
            Append_coordinates (c_displacement, i, j)
        else:

            
    return 

    

In [ ]:
def Navigation_in_minigrid (M, x, y, n): # M is the total matrix image; x and y are the dimensions of the minigrid; n is the step for the movement  

    c_minigrid = []
    
    for i in tqdm (range(0, x, n)): # Movement of the section along the x and y axis
        for j in range (0, y, n):

            scanning_view = M[i-128+n:i+128+n, j-128+n:j+128+n]
            Append_coordinates(c_minigrid, i, j)
            
    return c_minigrid
            

In [ ]:
def Border_detection (border_right, border_left, border_up, border_down, dimension_right, dimension_down): 
    """
    This function detects borders when scanning the image.

    border_x and border_y: coordinates along the x and y axis that we want to verify if they are exceeding the limits of the images.
    
    """
    if border_left < 0 or border_right >= dimension_right:
        return True
 
    if border_up > 0 or border_down >= dimension_down : 
        return True

    
    

In [ ]:
def Draw_Tracking_Line (image_copy, coordinates_list):
    """
    This function draw a tracking line of the movement.

    """
    #Iterating over pairs of points in the coordinates list to draw the line
    for start_point, end_point in zip(coordinates_list[:-1], coordinates_list[1:]): 
        cv2.line(image_copy, start_point, end_point, color, thickness)

    return image_copy

In [ ]:
def Draw_Coordinates (image_copy, coordinates_list):
    """
    This function draw the coordinates on a image
    """
    for point in coodinates_list:
        cv2.line(image_copy, point, 5,red, -1) )

    return image_copy

In [ ]:
def Append_coordinates (coordinates_list, i, j):
    
    coordinates_list.append((i, j))
    
    return coordinates_list

In [3]:
def border(self, i, j):
        """
        This function detects borders when scanning the image.
        border_x and border_y: coordinates along the x and y axis that we want to verify if they are exceeding the limits of the images.
        
        """
        
        border_left = j
        border_right = j + self.fov
        border_up = i - self.fov
        border_down = i 
        dimension_down, dimension_right = self.image_sample.shape
        
        if border_left >= 0 or border_right < dimension_right:
            return True
    
        if border_up >= 0 or border_down < dimension_down : 
            return True


def find_np(self, p_np, coord_microscope):
        """
        This algorithm determines the navigation path for detecting nanoparticles. The direction of movement is guided by the probability of encountering nanoparticles at the next 
        step. This probability is provided by our model, which compares the likelihood of finding nanoparticles in different directions and selects the direction with the highest 
        probability.
        """
        n = self.step
        i, j = coord_microscope

        #Up
        if not self.border(i - n, j): 
            scanning_m_u = self.image_sample[i-self.fov-n:i-n, j:j+self.fov]
            prob_mov_u = self.calculate_np_probability(scanning_m_u)
        else:
            prob_mov_u = 0
        
        #Down
        if not self.border(i+n, j): 
            scanning_m_d = self.image_sample[i-self.fov+n:i+n, j:j+self.fov]
            prob_mov_d = self.calculate_np_probability(scanning_m_d)
        else: 
            prob_mov_d = 0

        #Left
        if not self.border(i, j-n): 
            scanning_m_r = self.image_sample[i-self.fov:i, j+n:j+self.fov+n]
            prob_mov_r = self.calculate_np_probability(scanning_m_r)
        else:
            prob_mov_l = 0
        
        #Right
        if not self.border(i, j+n):     
            scannig_m_l = self.image_sample[i-self.fov:i, j-n:j+self.fov-n]
            prob_mov_l = self.calculate_np_probability(scannig_m_l)
        else:
            prob_mov_r = 0
        
        probs = [prob_mov_r, prob_mov_l, prob_mov_u, prob_mov_d]
        max_prob = np.max(probs)

        #Acquiring the image of the nanoparticle when the surroundings haven't higher porbability of finding a nanoparticle than the current posistion
        if max_prob <= p_np: 
            i_new, j_new = i, j 
        
        #Displacement
        if max_prob == prob_mov_r: #go to the right
            i_new, j_new = i, j+n
            
        elif max_prob == prob_mov_l: #go to the left
            i_new, j_new = i, j-n
   
        elif max_prob == prob_mov_u: #go up
            i_new, j_new = i-n, j
                
        elif max_prob == prob_mov_d: #go down
            i_new, j_new = i+n, j

        return i_new, j_new
